In [2]:
from transformers import pipeline
from pprint import pprint
import mlflow


/anaconda/envs/rest-api-demo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
classifier_path = "./models/pretrained/roberta-base-go_emotions"
summarizer_path = './models/pretrained/t5_small_text_summarization'

### 1. Download and save the huggingface models

In [4]:
# classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)
# classifier.save_pretrained('./models/pretrained/roberta-base-go_emotions')

In [5]:
loaded_classifier = pipeline(task="text-classification", 
                             model=classifier_path, 
                             top_k=None)

In [6]:
results = loaded_classifier.predict(['I love football', 'I hate hot weather'])
results

[[{'label': 'love', 'score': 0.950127363204956},
  {'label': 'admiration', 'score': 0.03602135181427002},
  {'label': 'approval', 'score': 0.016779204830527306},
  {'label': 'joy', 'score': 0.01269887387752533},
  {'label': 'neutral', 'score': 0.00947537925094366},
  {'label': 'gratitude', 'score': 0.007057229056954384},
  {'label': 'optimism', 'score': 0.0052270712330937386},
  {'label': 'annoyance', 'score': 0.004698918666690588},
  {'label': 'desire', 'score': 0.004633420147001743},
  {'label': 'excitement', 'score': 0.0045125894248485565},
  {'label': 'realization', 'score': 0.004273843020200729},
  {'label': 'disapproval', 'score': 0.004228235222399235},
  {'label': 'sadness', 'score': 0.004081507213413715},
  {'label': 'anger', 'score': 0.004016295075416565},
  {'label': 'disappointment', 'score': 0.003941954579204321},
  {'label': 'amusement', 'score': 0.0030699302442371845},
  {'label': 'caring', 'score': 0.002992184367030859},
  {'label': 'confusion', 'score': 0.00290683144703

In [7]:
# summarizer = pipeline("summarization", model="Falconsai/text_summarization")
# summarizer.save_pretrained('./models/pretrained/t5_small_text_summarization')

In [8]:
loaded_summarizer = pipeline("summarization", 
                             model=summarizer_path)

In [10]:
ARTICLE = """ 
Pro-democracy media tycoon Jimmy Lai is facing a Hong Kong court in a long-awaited trial over charges that he "colluded with foreign forces".
Mr Lai, 76, denies the charges. Jailed since December 2020, he faces life in prison if he is found guilty.
He was arrested under the National Security Law, which China has been accused of wielding to crush dissent.
His case has caused international uproar and is widely seen as a test of Hong Kong's judicial independence.
Mr Lai, who appeared in court on Monday flanked by prison guards, is one of more than 250 activists, lawmakers and protesters who have been detained under National Security Law (NSL) and sedition charges since 2020. The trial, which has been delayed for a year, is expected to last for about 80 days.
Beijing, which introduced the NSL in 2020 in response to massive pro-democracy protests, insists that the law is necessary to quell unrest. It considers Mr Lai a traitor who sought to undermine China's national security. But critics say Mr Lai's case is yet another example of Beijing's tightening grip on Hong Kong.
"""

summary = loaded_summarizer(ARTICLE, max_length=230, min_length=30, do_sample=False)

In [11]:
summary

[{'summary_text': "Jimmy Lai, 76, faces life in prison if found guilty . He was arrested under the National Security Law . His case has caused international uproar and is widely seen as a test of Hong Kong's independence ."}]

### 2. Register the models to Azure ML's MLFlow server

In [ ]:
# import mlflow

In [ ]:
# experiment_name = 'register_pretained_transformer_models'
# mlflow.set_experiment(experiment_name)

2023/12/18 08:16:48 INFO mlflow.tracking.fluent: Experiment with name 'register_pretained_transformer_models' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1702887409034, experiment_id='55795ceb-a274-41f1-a25f-dac4ceb8567f', last_update_time=None, lifecycle_stage='active', name='register_pretained_transformer_models', tags={}>

In [ ]:
# with mlflow.start_run() as run:
#     run_id = run.info.run_id
#     print(f"started run name: {run.info.run_name}")
#     print(f"started run id: {run.info.run_id}")
#     # artifact path is the folder structure inside MLFlow server you want to organize the model into, not the local folder
#     mlflow.transformers.log_model(transformers_model=loaded_classifier,
#                                   artifact_path = "./models/pretrained/roberta-base-go_emotions",
#                                   registered_model_name = "roberta-base-go_emotions") 
#     mlflow.transformers.log_model(transformers_model=loaded_summarizer,
#                                   artifact_path = "./models/pretrained/t5_small_text_summarization",
#                                   registered_model_name = "t5_small_text_summarization")
# mlflow.end_run()

started run name: eager_dinner_b7sbs87v
started run id: 371d5d4d-98bd-4522-8e1a-9d81bae83a8d


/tmp/ipykernel_40179/735897364.py:6: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.35.2``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  mlflow.transformers.log_model(transformers_model=loaded_classifier,
/anaconda/envs/rest-api-demo/lib/python3.11/site-packages/mlflow/models/model.py:619: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.35.2``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
2023/12/18 08:36:42 WARNING mlflow.transformers: The model card could not be retrieved from the hub due to [Errno 21] Is a directory: 'models/pretrained/roberta-base-go_emotions'
/anaconda/en

### 3. Querying and loading registered models

As the environment that this notebook is running on is native Azure ML compute, there is no need to specify any credentials to initialize the mflow tracking client

In [12]:
client = mlflow.tracking.MlflowClient()

In [36]:
for model in client.search_registered_models():
    print(f"{model.name}")

azureml_AutoML_d9ec2959-d835-4011-ba19-28cc889ade79_0_output_mlflow_log_model_380109068
azureml_AutoML_d9ec2959-d835-4011-ba19-28cc889ade79_1_output_mlflow_log_model_380109068
azureml_AutoML_d9ec2959-d835-4011-ba19-28cc889ade79_2_output_mlflow_log_model_1072233275
azureml_AutoML_d9ec2959-d835-4011-ba19-28cc889ade79_3_output_mlflow_log_model_380109068
azureml_AutoML_d9ec2959-d835-4011-ba19-28cc889ade79_4_output_mlflow_log_model_380109068
azureml_AutoML_d9ec2959-d835-4011-ba19-28cc889ade79_5_output_mlflow_log_model_1072233275
azureml_AutoML_d9ec2959-d835-4011-ba19-28cc889ade79_6_output_mlflow_log_model_1047705112
azureml_AutoML_d9ec2959-d835-4011-ba19-28cc889ade79_7_output_mlflow_log_model_417421593
azureml_AutoML_d9ec2959-d835-4011-ba19-28cc889ade79_20_output_mlflow_log_model_56851338
azureml_AutoML_d9ec2959-d835-4011-ba19-28cc889ade79_21_output_mlflow_log_model_380109068
azureml_830f1b23-29de-4bd4-9b94-6ea1e07dd86e_output_mlflow_log_model_1655100396
ElasticnetWineModel
azureml_a13a595f

In [55]:
classifier_name = "roberta-base-go_emotions"
summarizer_name = "t5_small_text_summarization"

# aml_classifier = client.get_model_version(classifier_name, version=1)
# aml_summarizer = client.get_model_version(summarizer_name, version=1) # if the version is not specified, TypeError is thrown, latest version is not retrieved
# uri = client.get_model_version_download_uri(classifier_name, 1)
aml_classifier = mlflow.transformers.load_model('azureml://southeastasia.api.azureml.ms/mlflow/v2.0/subscriptions/b10dba21-0331-4510-a7cb-924077c9cb52/resourceGroups/dev-csdw-rg/providers/Microsoft.MachineLearningServices/workspaces/dev-csdw-ml-workspace/experiments/55795ceb-a274-41f1-a25f-dac4ceb8567f/runs/371d5d4d-98bd-4522-8e1a-9d81bae83a8d/artifacts/models/pretrained/roberta-base-go_emotions',
                                                top_k=None)
aml_summarizer = mlflow.transformers.load_model('azureml://southeastasia.api.azureml.ms/mlflow/v2.0/subscriptions/b10dba21-0331-4510-a7cb-924077c9cb52/resourceGroups/dev-csdw-rg/providers/Microsoft.MachineLearningServices/workspaces/dev-csdw-ml-workspace/experiments/55795ceb-a274-41f1-a25f-dac4ceb8567f/runs/371d5d4d-98bd-4522-8e1a-9d81bae83a8d/artifacts/models/pretrained/t5_small_text_summarization')

/tmp/ipykernel_51208/4178330736.py:7: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.35.2``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  aml_classifier = mlflow.transformers.load_model('azureml://southeastasia.api.azureml.ms/mlflow/v2.0/subscriptions/b10dba21-0331-4510-a7cb-924077c9cb52/resourceGroups/dev-csdw-rg/providers/Microsoft.MachineLearningServices/workspaces/dev-csdw-ml-workspace/experiments/55795ceb-a274-41f1-a25f-dac4ceb8567f/runs/371d5d4d-98bd-4522-8e1a-9d81bae83a8d/artifacts/models/pretrained/roberta-base-go_emotions',
2023/12/18 09:30:09 WARNING mlflow.transformers: Could not specify device parameter for this pipeline type
/tmp/ipykernel_51208/4178330736.py:9: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  

In [16]:
SAMPLE_ARTICLE = """ 
More training for door staff and funding for testing kits are among a package of measures announced to crack down on spiking.
The Home Office has said the law will also be "modernised" to make it clear spiking - putting alcohol or drugs into another person's drink or body without their consent - is a crime.
The plans have broadly been welcomed by campaigners.
But they stop short of calls from some to make spiking a specific offence.
The measures announced by the government include:
Training hundreds more door staff to stop potential perpetrators and signs someone may have been spiked
Investing in research into testing kits to detect if someone's drink has been spiked
Intensive operations by the National Police Chiefs' Council (NPCC) across England and Wales, targeting key weeks when spiking tends to be prevalent
An online tool to be rolled out to all police forces to make it easier for people to report spiking anonymously
Home Secretary James Cleverly said: "Spiking is a perverse crime which can have a lasting impact on victims.
"Our comprehensive new measures are designed to help police and staff in bars, restaurants, pubs and other premises to protect victims and bring more offenders to justice."
"""

In [44]:
aml_summary = aml_summarizer(SAMPLE_ARTICLE)
aml_summary

[{'summary_text': 'The Home Office has said the law will also be "modernised" to make it clear spiking is a crime . The plans have broadly been welcomed by campaigners . But they stop short of calls from some to make the crime a specific offence .'}]

In [56]:
predictions = aml_classifier.predict(aml_summary[0]['summary_text'])
predictions

[[{'label': 'approval', 'score': 0.7034388184547424},
  {'label': 'neutral', 'score': 0.30123162269592285},
  {'label': 'admiration', 'score': 0.041029393672943115},
  {'label': 'realization', 'score': 0.0273037850856781},
  {'label': 'optimism', 'score': 0.021668855100870132},
  {'label': 'disapproval', 'score': 0.01637597382068634},
  {'label': 'love', 'score': 0.005554392468184233},
  {'label': 'annoyance', 'score': 0.004261727910488844},
  {'label': 'joy', 'score': 0.0035044108517467976},
  {'label': 'disappointment', 'score': 0.003361587645485997},
  {'label': 'gratitude', 'score': 0.003002336947247386},
  {'label': 'desire', 'score': 0.002980227814987302},
  {'label': 'confusion', 'score': 0.002826269716024399},
  {'label': 'relief', 'score': 0.002815544605255127},
  {'label': 'caring', 'score': 0.002804043935611844},
  {'label': 'excitement', 'score': 0.002140974160283804},
  {'label': 'pride', 'score': 0.0020365267992019653},
  {'label': 'curiosity', 'score': 0.0009850237984210

In [45]:
uri = client.get_model_version_download_uri(classifier_name, 1)
uri

'azureml://southeastasia.api.azureml.ms/mlflow/v2.0/subscriptions/b10dba21-0331-4510-a7cb-924077c9cb52/resourceGroups/dev-csdw-rg/providers/Microsoft.MachineLearningServices/workspaces/dev-csdw-ml-workspace/experiments/55795ceb-a274-41f1-a25f-dac4ceb8567f/runs/371d5d4d-98bd-4522-8e1a-9d81bae83a8d/artifacts/./models/pretrained/roberta-base-go_emotions'

In [60]:
client._registry_uri

'azureml://southeastasia.api.azureml.ms/mlflow/v1.0/subscriptions/b10dba21-0331-4510-a7cb-924077c9cb52/resourceGroups/dev-csdw-rg/providers/Microsoft.MachineLearningServices/workspaces/dev-csdw-ml-workspace'

In [61]:
mlflow.search_experiments()

[<Experiment: artifact_location='', creation_time=1687485405015, experiment_id='280bb9ea-1e8f-433b-93c2-c2c9df44e1e0', last_update_time=None, lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='', creation_time=1693372122301, experiment_id='69e1afdb-0e0a-44ea-aec2-5de677076160', last_update_time=None, lifecycle_stage='active', name='100_pod_prediction_automl', tags={}>,
 <Experiment: artifact_location='', creation_time=1693461153794, experiment_id='799e6879-7fcd-4090-8d24-39936371d1d8', last_update_time=None, lifecycle_stage='active', name='all_pod_prediction_automl', tags={}>,
 <Experiment: artifact_location='', creation_time=1693791579521, experiment_id='798ef9fe-0902-4dc9-a22f-7c3ecb84a296', last_update_time=None, lifecycle_stage='active', name='notebook-runs', tags={}>,
 <Experiment: artifact_location='', creation_time=1693813436022, experiment_id='60df152c-8e9f-4619-a642-00797a44f6f5', last_update_time=None, lifecycle_stage='active', name='latest_a